In [1]:
import qiskit_braket_provider

In [2]:
#This cell has my index 0 qc from qiskit
from numpy import *
from qiskit import *
from math import *
from numpy import *
import networkx as nx
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.circuit.library import QFT
#
def controlled_unitary(qc, qubits: list, phases: list): # x,y,z = Specific Qubit; a,b,c,d = Phases
    qc.cp(phases[2]-phases[0], qubits[0], qubits[1]) # controlled-U1(c-a)
    qc.p(phases[0], qubits[0]) # U1(a)
    qc.cp(phases[1]-phases[0], qubits[0], qubits[2]) # controlled-U1(b-a)
    
    # controlled controlled U1(d-c+a-b)
    qc.cp((phases[3]-phases[2]+phases[0]-phases[1])/2, qubits[1], qubits[2])
    qc.cx(qubits[0], qubits[1])
    qc.cp(-(phases[3]-phases[2]+phases[0]-phases[1])/2, qubits[1], qubits[2])
    qc.cx(qubits[0], qubits[1])
    qc.cp((phases[3]-phases[2]+phases[0]-phases[1])/2, qubits[0], qubits[2])

def U(times, qc, unit, eigen, phases: list): # a,b,c = phases for U1; d,e,f = phases for U2; g,h,i = phases for U3; j,k,l = phases for U4; m_list=[m, n, o, p, q, r, s, t, u, a, b, c, d, e, f, g, h, i, j, k, l]
    controlled_unitary(qc, [unit[0]]+eigen[0:2], [0]+phases[0:3])
    controlled_unitary(qc, [unit[0]]+eigen[2:4], [phases[3]]+[0]+phases[4:6])
    controlled_unitary(qc, [unit[0]]+eigen[4:6], phases[6:8]+[0]+[phases[8]])
    controlled_unitary(qc, [unit[0]]+eigen[6:8], phases[9:12]+[0])
    
def final_U(times, eigen, phases: list):
    unit = QuantumRegister(1, 'unit')
    qc = QuantumCircuit(unit, eigen)
    for _ in range(2**times):
        U(times, qc, unit, eigen, phases)
    return qc.to_gate(label='U'+'_'+(str(2**times)))

# Storing the eigenvalues in a list
eigen_values = ["11000110", "10000111", "10001101", "01001110", "11001001", "01001011"]
#index 0 is 1-2-3-4
#index 1 is 1-2-4-3
#index 2 is 1-4-2-3
#index 3 is 1-4-3-2
#index 4 is 1-3-2-4
#index 5 is 1-3-4-2

def which_eigen(index):
    if index == 0:
        return "1-2-3-4"
    elif index == 1:
        return "1-2-4-3"
    elif index == 2:
        return "1-4-2-3"
    elif index == 3:
        return "1-4-3-2"
    elif index == 4:
        return "1-3-2-4"
    else:
        return "1-3-4-2"

# Function to place appropriate corresponding gate according to eigenstates
def eigenstates(qc, eigen_list, index):
    for i in range(0, len(eigen_list)):
        if eigen_values[index][i] == '1':
            qc.x(eigen_list[i])
        if eigen_values[index][i] == '0':
            pass
    qc.barrier()
    print("Chosen Eigen value is:", eigen_values[index], "which is", which_eigen(index))
    return qc

#Initializing the values for the array
x = (100.69*pi)/2
print(x)
phases = [(1/160.48)*x, (1/349.44)*x, (1/100.69)*x, (1/160.48)*x, (1/196.87)*x, (1/113.23)*x, (1/349.44)*x, (1/196.87)*x, (1/268.24)*x, (1/100.69)*x, (1/113.23)*x, (1/268.24)*x]
##############INDEX 0##########################
# Creating the 14 qubit circuit, 6 are for the qft and the rest are for the eigenvectors/QPE
unit = QuantumRegister(6, 'unit')
eigen = QuantumRegister(8, 'eigen')
unit_classical = ClassicalRegister(6, 'unit_classical')
qc = QuantumCircuit(unit, eigen, unit_classical)
#

# Setting one eigenstate, puts not gates into circuit  
eigenstates(qc, eigen, 0)
#

# Hadamard on the 'unit' qubits
qc.h(unit[:])
qc.barrier()
#

# Controlled Unitary  
for i in range(0, 6):
    qc.append(final_U(i, eigen, phases), [unit[5-i]] + eigen[:])
#

# Inverse QFT 
qc.barrier()
qft = QFT(num_qubits=len(unit), inverse=True, insert_barriers=True, do_swaps=False, name='QFT^-1')
qc.append(qft, qc.qubits[:len(unit)])
qc.barrier()
#

# Measure
qc.measure(unit, unit_classical)
#


158.16348214497813
Chosen Eigen value is: 11000110 which is 1-2-3-4


In [8]:
from qiskit_braket_provider import AWSBraketProvider

provider = AWSBraketProvider()

# devices
ionq_device = provider.get_backend("IonQ Device")

NoRegionError: You must specify a region.